<a href="https://colab.research.google.com/github/jjung2-oxy/Natural-Lang-Processing/blob/main/NLP_Homework_3_(Lyric_Generation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
from collections import defaultdict
import nltk
import string
from sklearn.model_selection import train_test_split
import io
import sys
import numpy as np
import random

import gensim
from gensim.models import Word2Vec
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding

In [ ]:
# Import Lyrics
country_lyrics = { "Song Name": [], "Lyrics": []}
metal_lyrics = { "Song Name": [], "Lyrics": []}
pop_lyrics = { "Song Name": [], "Lyrics": []}
rock_lyrics = { "Song Name": [], "Lyrics": []}

# Get Country Lyrics
country_directory = r'/content/drive/MyDrive/Homework3-Data/Homework 3 Data/Country'

for file in os.listdir(country_directory):
    f = os.path.join(country_directory, file)
    filename = os.path.splitext(file)[0]

    if os.path.isfile(f) and file.endswith('.txt'):
      with open(f, encoding='utf-8') as text:
          contents = text.read()
          country_lyrics["Song Name"].append(filename)
          country_lyrics["Lyrics"].append(contents)

# Turn dict into DataFrame, print first 5 songs
country_df = pd.DataFrame(country_lyrics)
country_df.head(5)

# Get Metal Lyrics
metal_directory = r'/content/drive/MyDrive/Homework3-Data/Homework 3 Data/Metal'

for file in os.listdir(metal_directory):
    f = os.path.join(metal_directory, file)
    filename = os.path.splitext(file)[0]

    if os.path.isfile(f) and file.endswith('.txt'):
      with open(f, encoding='utf-8') as text:
          contents = text.read()
          metal_lyrics["Song Name"].append(filename)
          metal_lyrics["Lyrics"].append(contents)

# Turn dict into DataFrame, print first 5 songs
metal_df = pd.DataFrame(metal_lyrics)
metal_df.head(5)

# Get Pop Lyrics
pop_directory = r'/content/drive/MyDrive/Homework3-Data/Homework 3 Data/Pop'

for file in os.listdir(pop_directory):
    f = os.path.join(pop_directory, file)
    filename = os.path.splitext(file)[0]

    if os.path.isfile(f) and file.endswith('.txt'):
      with open(f, encoding='utf-8') as text:
          contents = text.read()
          pop_lyrics["Song Name"].append(filename)
          pop_lyrics["Lyrics"].append(contents)

# Turn dict into DataFrame, print first 5 songs
pop_df = pd.DataFrame(pop_lyrics)
pop_df.head(5)

# Get Rock Lyrics
rock_directory = r'/content/drive/MyDrive/Homework3-Data/Homework 3 Data/Rock'

for file in os.listdir(rock_directory):
    f = os.path.join(rock_directory, file)
    filename = os.path.splitext(file)[0]

    if os.path.isfile(f) and file.endswith('.txt'):
      with open(f, encoding='utf-8') as text:
          contents = text.read()
          rock_lyrics["Song Name"].append(filename)
          rock_lyrics["Lyrics"].append(contents)

# Turn dict into DataFrame, print first 5 songs
rock_df = pd.DataFrame(rock_lyrics)
rock_df.head(5)
    

,Song Name,Lyrics
0,What's Up,25 years of my life and still\nI'm trying to g...
1,Sweet Child O' Mine,She's got a smile that it seems to me\nReminds...
2,T.N.T,See me ride out of the sunset\nOn your colour ...
3,Caught Up In You,I never knew there'd come a day\nWhen I'd be s...
4,Lucky,"Do you hear me,\nI'm talking to you\nAcross th..."


In [ ]:
# Data Pre-Processing
def clean_lyrics(x):
    x = x["Lyrics"]

    # Turn to lowercase
    x = x.lower()

    # Remove punctuation
    x = "".join([char for char in x if char not in string.punctuation])

    # Convert into Pandas series
    processed_lyrics = pd.Series(data=x)
    
    return processed_lyrics

# Pre-process country lyrics
processed_lyrics = country_df.apply(clean_lyrics, axis=1, result_type='expand')
country_df["Processed Lyrics"] = processed_lyrics
country_df.head(5)

# Pre-process metal lyrics
processed_lyrics = metal_df.apply(clean_lyrics, axis=1, result_type='expand')
metal_df["Processed Lyrics"] = processed_lyrics
metal_df.head(5)

# Pre-process pop lyrics
processed_lyrics = pop_df.apply(clean_lyrics, axis=1, result_type='expand')
pop_df["Processed Lyrics"] = processed_lyrics
pop_df.head(5)

# Pre-process rock lyrics
processed_lyrics = rock_df.apply(clean_lyrics, axis=1, result_type='expand')
rock_df["Processed Lyrics"] = processed_lyrics
rock_df.head(5)


,Song Name,Lyrics,Processed Lyrics
0,What's Up,25 years of my life and still\nI'm trying to g...,25 years of my life and still\nim trying to ge...
1,Sweet Child O' Mine,She's got a smile that it seems to me\nReminds...,shes got a smile that it seems to me\nreminds ...
2,T.N.T,See me ride out of the sunset\nOn your colour ...,see me ride out of the sunset\non your colour ...
3,Caught Up In You,I never knew there'd come a day\nWhen I'd be s...,i never knew thered come a day\nwhen id be say...
4,Lucky,"Do you hear me,\nI'm talking to you\nAcross th...",do you hear me\nim talking to you\nacross the ...


In [ ]:
# Indexes words in word_indices(word:index) and indices_word(index:word) format, and gets vocabulary (words) and number of unique words (vocab_size)

def generate_dict_data(text):
    word_to_index= dict() # Dict {word: integer}
    index_to_word = dict()  # Dict {integer: word}
    corpus = [] # All words in the lyrics
    
    # Create corpus of ALL words (not-unique)
    for row in text:
        lines = row.split('\n')

        for line in lines:
            # add newline as seperate word to each line 
            line  = line + " \n "
            for word in line.split(" "):
                if word != '':
                    corpus.append(word)
    print(corpus)

    # Create vocabulary of unique words from corpus
    print(f"Size of corpus: {len(corpus)}")
    words = set(corpus)
    vocab_size = len(words)
    print(f"Size of vocabulary: {vocab_size}")

    # Create word indices
    word_indices = dict((c, i) for i, c in enumerate(words))
    indices_word = dict((i, c) for i, c in enumerate(words))

    return word_indices, indices_word, words, vocab_size, corpus

In [ ]:
# Turns lyrics into list of lists of words [[lyric_word1, lyric_word2, lyric_word3,...], [lyric2_word1, lyric2_word2, ...], ...]
def lyrics_to_sentences(genre_df, corpus):
    lyrics_in_sentences = []  # All sentences in lyrics
    sentence = []   # One sentence in lyrics
    lyrics_in_words = [] # Tokenized (word by word) lyrics

    # Add word from corpus to list of sentences
    for word in corpus:
        if word == "\n":
            lyrics_in_words.append(word)
            sentence.append(word)
            # Add sentence to list of sentences
            lyrics_in_sentences.append(sentence)
            # Clear current sentence
            sentence = []
        elif word != '':
            sentence.append(word)
            lyrics_in_words.append(word)
            
    return lyrics_in_words, lyrics_in_sentences

# Turn text into sequences of sequence_length (default=5) words
def lyrics_to_sequence(genre_df, genre_corpus, sequence_length=5):
    sequences = []
    next_words = []

    # Turn lyrics into all words and into seperate sentences.
    lyrics_in_words, sentences = lyrics_to_sentences(genre_df, genre_corpus)

    # For genre lyrics in words, create list of sequence_length sequences of words and list of the next word in the sequence
    for i in range(0, len(lyrics_in_words) - sequence_length):
        sequences.append(lyrics_in_words[i: i + sequence_length])
        next_words.append(lyrics_in_words[i + sequence_length])

    print('Total sequences:', len(sequences))
    return sequences, next_words

# Split sequences and next_words into training/testing input/output.
def train_test(sequences, next_words):

    # sequences (input), next_words (ouput)
    sequences_train, sequences_test, next_words_train, next_words_test = train_test_split(sequences, next_words, test_size=0.2, random_state=42)
    return sequences_train, sequences_test, next_words_train, next_words_test


In [ ]:
# Create sequential model with word embedding layer and LSTM layer. Dropout to prevent overfitting. 
def get_model(vocab_size, dropout=0.2):
    # Sequential model
    model = Sequential()

    # Embedding layer for word embedding
    model.add(Embedding(input_dim= vocab_size, output_dim=1024))

    # Bidirectional LSTM model with 128 nodes
    model.add(Bidirectional(LSTM(128)))

    # Dropout layer. Activated and set to 0.2 default.
    if dropout > 0:
        model.add(Dropout(dropout))

    # Dense layer
    model.add(Dense(vocab_size))

    # Softmax Activation
    model.add(Activation('softmax'))

    # Spare categorical b/c output is NOT one-hot encoded.
    model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

    return model

In [ ]:
# generator function from keras-team/keras/blob/master/examples/lstm_text_generation.py
# Feeds neural network batches of inputs/outputs to prevent insufficient memory issues.
def generator(sentence_list, next_word_list, batch_size, word_indices):
    index = 0
    SEQUENCE_LEN = 5

    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t] = word_indices[w]
      
            y[i] = word_indices[next_word_list[index % len(sentence_list)]]
            index = index + 1
        yield x, y

# Train model with sequences as input and next_words as output to guess
def fit_model(sequences_train, next_words_train, sequences_test, next_words_test, model, word_indices):
    BATCH_SIZE = 32

    # If loss stops improving after 3 epochs, stop learning early
    callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
  
    # Fit model. Use generator to feed neural network batches of input/output
    model.fit(generator(sequences_train, next_words_train, BATCH_SIZE, word_indices),
                    steps_per_epoch=int(len(sequences_train)/BATCH_SIZE) + 1,
                    epochs=20,  
                    callbacks=[callback],
                    validation_data=generator(sequences_test, next_words_test, BATCH_SIZE, word_indices),
                    validation_steps=int(len(sequences_test)/BATCH_SIZE) + 1)

    return model

In [ ]:
# sample function from keras-team/keras/blob/master/examples/lstm_text_generation.py
# Uses probability distributions to predict the next_word (doesn't choose best fitting word, chooses randomly while accounting for best fitting words)
def sample(preds, temperature=1.0):
    # Ensure list can be used as input
    preds = np.asarray(preds).astype('float64')

    # Puts predictions back into logspace. Temperature controls 'randomness' of model
    preds = np.log(preds) / temperature

    # Apply softmax
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    # Sample one-hot vector 
    probas = np.random.multinomial(1, preds, 1)
    
    # Returns index of largest vector which correpsonds to a word in our vocabulary
    return np.argmax(probas)

# Generates the first line of our lyric
def generate_lyrics(model, indices_word, word_indices, seed, sequence_length, vocabulary, diversity):
    sentence = seed
    vocabulary_length = len(vocabulary)

    # Prints the seed sentence as words
    for word in sentence:
        print(" "+word, end="")

    # Executes for 4 lines of generated lyrics. (5 total printed with above sentence)
    newline_count = 0
    while newline_count < 5:
        x_pred = np.zeros((1, sequence_length))

        for t, word in enumerate(sentence):
            x_pred[0, t] = word_indices[word]

        # Predict next word vector
        preds = model.predict(x_pred, verbose=0)[0]

        # Choose index from Conditional probability distribution
        next_index = sample(preds, diversity)

        # Find next_word by index
        next_word = indices_word[next_index]

        # Remove first word from sentence, add next_word
        sentence = sentence[1:]
        sentence.append(next_word)

        # Count number of newlines generated
        if next_word == '\n':
            newline_count = newline_count + 1

        # Print generated word
        print(" " + next_word, end="")
    print("\n")

    return next_word

def generate_seed(genre_sequences):
    # Generated random index
    seed_index = random.randint(0, int(len(genre_sequences) - 1))

    # Choose seed from genre sequences
    seed = genre_sequences[seed_index]

    # Take only the first 5 words of sequence       
    seed = seed[0:5]
        
    return seed

# Generate lyrics based on genre
def print_lyrics(genre_sequences, model, genre_itw, genre_wti, genre_words):
    
    # Find random 5 word sequence to base the generation on
    seed = generate_seed(genre_sequences)

    # Generate the lyrics
    generate_lyrics(model, genre_itw, genre_wti, seed, sequence_length=5, vocabulary=genre_words, diversity=.75)

In [ ]:
# PREPARE WORD INDEXES, WORD SENTENCES, AND SPLIT DATA INTO TRAINING AND TESTING

print("Preparing Country Lyrics:")
country_wti, country_itw, country_words, country_length, country_corpus = generate_dict_data(country_df["Processed Lyrics"])
country_sequences, country_nextwords = lyrics_to_sequence(country_df, country_corpus)
country_sequence_train, country_sequence_test, country_nextword_train, country_nextword_test = train_test(country_sequences, country_nextwords)
print(country_sequences)
print(country_nextwords)
print("\n")


print("Preparing Metal lyrics:")
metal_wti, metal_itw, metal_words, metal_length, metal_corpus = generate_dict_data(metal_df["Processed Lyrics"])
metal_sequences, metal_nextwords = lyrics_to_sequence(metal_df, metal_corpus)
metal_sequence_train, metal_sequence_test, metal_nextword_train, metal_nextword_test = train_test(metal_sequences, metal_nextwords)
print(metal_sequences)
print(metal_nextwords)
print("\n")

print("Preparing Pop lyrics:")
pop_wti, pop_itw, pop_words, pop_length, pop_corpus = generate_dict_data(pop_df["Processed Lyrics"])
pop_sequences, pop_nextwords = lyrics_to_sequence(pop_df, pop_corpus)
pop_sequence_train, pop_sequence_test, pop_nextword_train, pop_nextword_test = train_test(pop_sequences, pop_nextwords)
print(pop_sequences)
print(pop_nextwords)
print("\n")

print("Preparing Rock lyrics:")
rock_wti, rock_itw, rock_words, rock_length, rock_corpus = generate_dict_data(rock_df["Processed Lyrics"])
rock_sequences, rock_nextwords = lyrics_to_sequence(rock_df, rock_corpus)
rock_sequence_train, rock_sequence_test, rock_nextword_train, rock_nextword_test = train_test(rock_sequences, rock_nextwords)
print(rock_sequences)
print(rock_nextwords)
print("\n")


In [ ]:
print("CAUTION: LOADING MODELS DOES NOT WORK! NEWLY CREATED WORD INDICES WON'T BE ALIGNED TO LOADED MODEL...")
load_or_train_model = input("To train new models: 0, To load models: 1")

if load_or_train_model == '0':
    train_country = input("Train country? y/n")
    if train_country == "y":
        # COUNTRY MODEL CREATION/TRAINING/FITTING

        # Instantiate (build) model
        print("Building country model...")
        country_model_untrained = get_model(country_length)
        country_model_untrained.summary()

        # Fit (train) Model
        country_model = fit_model(country_sequence_train, country_nextword_train, country_sequence_test, country_nextword_test, country_model_untrained, country_wti)

        # Save Model
        country_model.save('/content/drive/MyDrive/Homework3-Data/Homework 3 Data/country_model')

    train_metal = input("Train metal? y/n")
    if train_metal == "y":
        # METAL MODEL CREATION/TRAINING/FITTING

        # Instantiate (build) model
        print("Building metal model...")
        metal_model_untrained = get_model(metal_length)
        metal_model_untrained.summary()

        # Fit (train) Model
        metal_model = fit_model(metal_sequence_train, metal_nextword_train, metal_sequence_test, metal_nextword_test, metal_model_untrained, metal_wti)

        # Save Model
        metal_model.save('/content/drive/MyDrive/Homework3-Data/Homework 3 Data/metal_model')

    train_pop = input("Train pop? y/n")
    if train_pop == "y":
        # POP MODEL CREATION/TRAININIG/FITTING

        # Instantiate (build) Model
        print("Building pop model...")
        pop_model_untrained = get_model(pop_length)
        pop_model_untrained.summary()

        # Fit (train) Model
        pop_model = fit_model(pop_sequence_train, pop_nextword_train, pop_sequence_test, pop_nextword_test, pop_model_untrained, pop_wti)

        # Save Model
        pop_model.save('/content/drive/MyDrive/Homework3-Data/Homework 3 Data/pop_model')

    train_rock = input("Train rock? y/n")
    if train_rock == 'y':
        # ROCK MODEL CREATION/TRAINING/FITTING

        # Instantiate (build) Model
        print("Building rock model...")
        rock_model_untrained = get_model(rock_length)
        rock_model_untrained.summary()

        # Fit (train) Model
        rock_model = fit_model(rock_sequence_train, rock_nextword_train, rock_sequence_test, rock_nextword_test, rock_model_untrained, rock_wti)

        # Save Model
        rock_model.save('/content/drive/MyDrive/Homework3-Data/Homework 3 Data/rock_model')

else:
    # Loading Models doesn't work for lyric generation... Pre-loaded Models won't be aligned with newly generated word indices...
    # Load Country Model
    print("Loading Country Model...")
    country_model = keras.models.load_model('/content/drive/MyDrive/Homework3-Data/Homework 3 Data/country_model')

    # Load Metal Model
    print("Loading Metal Model...")
    metal_model = keras.models.load_model('/content/drive/MyDrive/Homework3-Data/Homework 3 Data/metal_model')

    # Load Pop Model
    print("Loading Pop Model...")
    pop_model = keras.models.load_model('/content/drive/MyDrive/Homework3-Data/Homework 3 Data/pop_model')

    # Load Rock Model
    print("Loading Rock Model...")
    rock_model = keras.models.load_model('/content/drive/MyDrive/Homework3-Data/Homework 3 Data/rock_model')


In [ ]:
# Print Country Lyrics
print("Generating country lyrics...")
print_lyrics(country_sequences, country_model, country_itw, country_wti, country_words)

In [ ]:
# Print metal lyrics
print("Generating metal lyrics...")
metal_sentences = lyrics_to_words(metal_df)
print_lyrics(metal_sentences, metal_model, metal_itw, metal_wti, metal_words)

In [ ]:
# Print Pop lyrics
print("Generating pop lyrics...")
pop_sentences = lyrics_to_words(pop_df)
print_lyrics(pop_sentences, pop_model, pop_itw, pop_wti, pop_words)

In [ ]:
# Print rock lyrics
print("Generating rock lyrics...")
rock_sentences = lyrics_to_words(rock_df)
print_lyrics(rock_sentences, rock_model, rock_itw, rock_wti, rock_words)